In [1]:
from nltk.corpus import stopwords
import pandas as pd
import numpy as np
import csv
import os
import string
#
BASE_DIR = os.path.join( os.path.dirname(os.getcwd()))
print(BASE_DIR)
train_path = BASE_DIR + "\\data\\ssec-aggregated\\train-combined-0.0.csv"
test_path = BASE_DIR + "\\data\\ssec-aggregated\\test-combined-0.0.csv"
# Emotion Categories (Classes)
emotions = ("anger","anticipation","disgust","fear","joy","sadness","surprise","trust")
alpha = 1 # Laplace Smoothener

D:\Projects\ARI5121_NLP\75_Assignment_1


In [2]:
def open_file_and_return_dump(path):
    """ 
    Opens file and returns a dump containing all the data.
    The data structure used to record the dumped data is a list of strings,
    where each string represents a single record).
    """
    corpus = []
    with open(path) as csvfile:
        rows = csv.reader(csvfile)
        for row in rows:
            temp_string = ""
            for item in row:
                temp_string += str(item) + " "
            corpus.append(temp_string)
    return corpus
#
corpus = open_file_and_return_dump(path=train_path)
#
print("Shape of training corpus: \n" + str(np.shape(corpus)) + "\n")
print("First 5 lines of training corpus: \n" + str(corpus[0:5]))

Shape of training corpus: 
(2912,)

First 5 lines of training corpus: 
['Anger\tAnticipation\tDisgust\tFear\t---\tSadness\tSurprise\t---\t@tedcruz And  #HandOverTheServer she wiped clean + 30k deleted emails  explains dereliction of duty/lies re #Benghazi etc #tcot    ', '---\tAnticipation\t---\tFear\tJoy\t---\t---\tTrust\tHillary is our best choice if we truly want to continue being a progressive nation. #Ohio       ', "Anger\tAnticipation\tDisgust\t---\tJoy\tSadness\t---\t---\t@TheView I think our country is ready for a female pres  it can't ever be Hillary      ", "Anger\tAnticipation\tDisgust\tFear\t---\tSadness\t---\t---\tI just gave an unhealthy amount of my hard-earned money away to the big gov't & untrustworthy IRS. #WhyImNotVotingForHillary       ", '---\t---\t---\t---\tJoy\t---\t---\t---\t@PortiaABoulger Thank you for adding me to your list       ']


In [3]:
def split_labels_from_vectors(corpus):
    """
    Takes corpus as input, and splits the data into respective labels and training data.
    Returns this data as two separate lists
    """
    #
    label_count = len(emotions)
    character_splitter = '\t'
    train_y, train_X = [], []
    for row in corpus:
        groups = row.split(character_splitter)
        train_y.append(character_splitter.join(groups[:label_count]))
        train_X.append(character_splitter.join(groups[label_count:]))
    return train_y, train_X
#
train_y, train_X = split_labels_from_vectors(corpus=corpus)
print("First 5 lines of training corpus: \n" + str(train_X[0:5]) + "\n")
print("First 5 lines of training corpus: \n" + str(train_y[0:5]) + "\n")
print("Shape of training corpus: \n" + str(np.shape(train_X)))
print("Shape of training corpus: \n" + str(np.shape(train_y)))

First 5 lines of training corpus: 
['@tedcruz And  #HandOverTheServer she wiped clean + 30k deleted emails  explains dereliction of duty/lies re #Benghazi etc #tcot    ', 'Hillary is our best choice if we truly want to continue being a progressive nation. #Ohio       ', "@TheView I think our country is ready for a female pres  it can't ever be Hillary      ", "I just gave an unhealthy amount of my hard-earned money away to the big gov't & untrustworthy IRS. #WhyImNotVotingForHillary       ", '@PortiaABoulger Thank you for adding me to your list       ']

First 5 lines of training corpus: 
['Anger\tAnticipation\tDisgust\tFear\t---\tSadness\tSurprise\t---', '---\tAnticipation\t---\tFear\tJoy\t---\t---\tTrust', 'Anger\tAnticipation\tDisgust\t---\tJoy\tSadness\t---\t---', 'Anger\tAnticipation\tDisgust\tFear\t---\tSadness\t---\t---', '---\t---\t---\t---\tJoy\t---\t---\t---']

Shape of training corpus: 
(2912,)
Shape of training corpus: 
(2912,)


In [4]:
def tokenize_corpus(data, delimeter=" "):
    """
    Takes a list as input, and tokenizes the data.
    """
    corpus = []
    [(corpus.append(row.split(delimeter))) for row in data]
    return corpus
tokenized_train_X, tokenized_train_y = tokenize_corpus(data=train_X), tokenize_corpus(data=train_y, delimeter="\t")
#
print("First 5 lines of training corpus: \n" + str(tokenized_train_X[0:5]) + "\n")
print("First 5 lines of training corpus: \n" + str(tokenized_train_y[0:5]) + "\n")
print("Shape of training corpus: \n" + str(np.shape(tokenized_train_X)))
print("Shape of training corpus: \n" + str(np.shape(tokenized_train_y)))

First 5 lines of training corpus: 
[['@tedcruz', 'And', '', '#HandOverTheServer', 'she', 'wiped', 'clean', '+', '30k', 'deleted', 'emails', '', 'explains', 'dereliction', 'of', 'duty/lies', 're', '#Benghazi', 'etc', '#tcot', '', '', '', ''], ['Hillary', 'is', 'our', 'best', 'choice', 'if', 'we', 'truly', 'want', 'to', 'continue', 'being', 'a', 'progressive', 'nation.', '#Ohio', '', '', '', '', '', '', ''], ['@TheView', 'I', 'think', 'our', 'country', 'is', 'ready', 'for', 'a', 'female', 'pres', '', 'it', "can't", 'ever', 'be', 'Hillary', '', '', '', '', '', ''], ['I', 'just', 'gave', 'an', 'unhealthy', 'amount', 'of', 'my', 'hard-earned', 'money', 'away', 'to', 'the', 'big', "gov't", '&', 'untrustworthy', 'IRS.', '#WhyImNotVotingForHillary', '', '', '', '', '', '', ''], ['@PortiaABoulger', 'Thank', 'you', 'for', 'adding', 'me', 'to', 'your', 'list', '', '', '', '', '', '', '']]

First 5 lines of training corpus: 
[['Anger', 'Anticipation', 'Disgust', 'Fear', '---', 'Sadness', 'Surprise

In [5]:
def _replace(word, symbols, placeholder=""):
    """
    An overriding of the original python method, so as to replace all characters 
    in a string based on whether they occur in a list.
    """
    temp_string = word
    for symbol in symbols:
        temp_string = temp_string.replace(symbol, placeholder)
    return temp_string
#
def remove_stop_words(data):
    """
    Takes a list of data, and iterates over each element to 
    scan and remover stop words.
    
    The method ensures to convert all text instances to lowercase.
    
    The method ensures to remove jargon symbols (#,%,&,etc).
    
    This method ensures to remove punctuation.
    """
    temp_list, jargon_symbols = [], ('%','$','#','@','^','&','*','(',')','+','-','/','\'','!','?','.',',',':',';','~','0','1','2','3','4','5','6','7','8','9')
    for row in data:
        filtered_words = [_replace(word.lower().translate(string.punctuation), symbols=jargon_symbols) for word in row if word.lower() not in stopwords.words('english')]          
        temp_list.append(filtered_words)
    #
    new_list = []
    for row in temp_list:
        row_list = []
        for word in row:
            if word != "":
                row_list.append(word)
        new_list.append(row_list)
    #
    return new_list
#
cleaned_tokenized_train_X, cleaned_tokenized_train_y = remove_stop_words(data=tokenized_train_X), remove_stop_words(data=tokenized_train_y)
print("First 5 lines of training corpus: \n" + str(cleaned_tokenized_train_X[0:5]) + "\n")
print("First 5 lines of training corpus: \n" + str(cleaned_tokenized_train_y[0:5]) + "\n")
print("Shape of training corpus: \n" + str(np.shape(cleaned_tokenized_train_X)))
print("Shape of training corpus: \n" + str(np.shape(cleaned_tokenized_train_y)))

First 5 lines of training corpus: 
[['tedcruz', 'handovertheserver', 'wiped', 'clean', 'k', 'deleted', 'emails', 'explains', 'dereliction', 'dutylies', 'benghazi', 'etc', 'tcot'], ['hillary', 'best', 'choice', 'truly', 'want', 'continue', 'progressive', 'nation', 'ohio'], ['theview', 'think', 'country', 'ready', 'female', 'pres', 'cant', 'ever', 'hillary'], ['gave', 'unhealthy', 'amount', 'hardearned', 'money', 'away', 'big', 'govt', 'untrustworthy', 'irs', 'whyimnotvotingforhillary'], ['portiaaboulger', 'thank', 'adding', 'list']]

First 5 lines of training corpus: 
[['anger', 'anticipation', 'disgust', 'fear', 'sadness', 'surprise'], ['anticipation', 'fear', 'joy', 'trust'], ['anger', 'anticipation', 'disgust', 'joy', 'sadness'], ['anger', 'anticipation', 'disgust', 'fear', 'sadness'], ['joy']]

Shape of training corpus: 
(2912,)
Shape of training corpus: 
(2912,)


In [6]:
class NaiveBayes:
    """
    Implements a hand built Naive Bayes superivised classifier on top of the BOW methodology,
    along with predict and evaluation methods for the respective model built.
    """
    #
    def __init__(self, emotions, emotion_index):
        self.BOW = {}
        self.emotions = emotions
        self.emotion_index = emotion_index
        if self.emotion_index > len(self.emotions):
            print("Index exceeds the current emotion list capacity. Index musn't exceed " + str(len(self.emotions)))
    #
    def __prod(self,iterable):
        """
        Product multiplier
        """
        total = 1
        for number in iterable:
            total *= number
        return total
    #
    def fit(self,X,y):
        """
        NB classifier function used to train the model. 
        Operates by constructing a bag of words model.
        """
        selected_emotion = self.get_sentiment()
        for i in range(len(y)):
            if selected_emotion in y[i]:
                for word in X[i]:
                    if word in self.BOW:
                        self.BOW[word] += 1
                    else:
                        self.BOW[word] = 1                
    #
    def predict_proba(self,X):
        """
        NB classifier function used to return probability estimates for the test vector.
        """
        #
        likelihood_probalities = []
        for word in X:
            if word in self.BOW:
                likelihood_probalities.append((self.BOW[word]/len(self.BOW.values())) + alpha)
            else:
                likelihood_probalities.append(alpha)
        likelihood_prob = self.__prod(likelihood_probalities)
        class_prior_prob = 1/len(self.emotions)
        return likelihood_prob * class_prior_prob
    #
    def get_sentiment(self):
        """
        Returns sentiment of this instance classifier
        """
        return self.emotions[self.emotion_index]
#
NB_classifier_anger = NaiveBayes(emotions, 0)
NB_classifier_anger.fit(X=cleaned_tokenized_train_X, y=cleaned_tokenized_train_y)
NB_classifier_anticipation = NaiveBayes(emotions, 1)
NB_classifier_anticipation.fit(X=cleaned_tokenized_train_X, y=cleaned_tokenized_train_y)
NB_classifier_disgust = NaiveBayes(emotions, 2)
NB_classifier_disgust.fit(X=cleaned_tokenized_train_X, y=cleaned_tokenized_train_y)
NB_classifier_fear = NaiveBayes(emotions, 3)
NB_classifier_fear.fit(X=cleaned_tokenized_train_X, y=cleaned_tokenized_train_y)
NB_classifier_joy = NaiveBayes(emotions, 4)
NB_classifier_joy.fit(X=cleaned_tokenized_train_X, y=cleaned_tokenized_train_y)
NB_classifier_sadness = NaiveBayes(emotions, 5)
NB_classifier_sadness.fit(X=cleaned_tokenized_train_X, y=cleaned_tokenized_train_y)
NB_classifier_surprise = NaiveBayes(emotions, 6)
NB_classifier_surprise.fit(X=cleaned_tokenized_train_X, y=cleaned_tokenized_train_y)
NB_classifier_trust = NaiveBayes(emotions, 7)
NB_classifier_trust.fit(X=cleaned_tokenized_train_X, y=cleaned_tokenized_train_y)
classifiers = [NB_classifier_anger, 
               NB_classifier_anticipation, 
               NB_classifier_disgust, 
               NB_classifier_fear, 
               NB_classifier_joy, 
               NB_classifier_sadness, 
               NB_classifier_surprise, 
               NB_classifier_trust]
test = ['senate','liberty','reprojustice']
pred = NB_classifier_anger.predict_proba(test)
print(pred)
pred = NB_classifier_anticipation.predict_proba(test)
print(pred)
pred = NB_classifier_disgust.predict_proba(test)
print(pred)
pred = NB_classifier_fear.predict_proba(test)
print(pred)
pred = NB_classifier_joy.predict_proba(test)
print(pred)
pred = NB_classifier_sadness.predict_proba(test)
print(pred)
pred = NB_classifier_surprise.predict_proba(test)
print(pred)
pred = NB_classifier_trust.predict_proba(test)
print(pred)

0.12511794306503593
0.1251923368853877
0.12511855532600463
0.12513436813629975
0.12521545760223646
0.1251622981235286
0.12512322014743035
0.12523695979551033


In [7]:
#
# Testing
test_corpus = open_file_and_return_dump(path=test_path)
test_y, test_X = split_labels_from_vectors(corpus=test_corpus)
tokenized_test_X, tokenized_test_y = tokenize_corpus(data=test_X), tokenize_corpus(data=test_y, delimeter="\t")
cleaned_tokenized_test_X, cleaned_tokenized_test_y = remove_stop_words(data=tokenized_test_X), remove_stop_words(data=tokenized_test_y)
#
print("First 5 lines of training corpus: \n" + str(cleaned_tokenized_test_X[0:5]) + "\n")
print("First 5 lines of training corpus: \n" + str(cleaned_tokenized_test_y[0:5]) + "\n")
print("Shape of training corpus: \n" + str(np.shape(cleaned_tokenized_test_X)))
print("Shape of training corpus: \n" + str(np.shape(cleaned_tokenized_test_y)) + "\n\n--------------------------------------\n")
#
def predict_overall_sentiment(classifiers, X):
    """
    Accepts a list of NB classifiers, and returns the highest probabilistic sentiment
    """
    sentiment, prob = 0, 0
    for classifier in classifiers:
        temp_prob = classifier.predict_proba(X=X)
        if temp_prob > prob:
            sentiment = classifier.get_sentiment()
            prob = temp_prob
    return sentiment, prob
#
y_true, y_pred = [], []
for i in range(len(cleaned_tokenized_test_X)):
    sentiment, prob = predict_overall_sentiment(classifiers=classifiers, X=cleaned_tokenized_test_X[i])
    y_pred.append(sentiment)
    y_true.append(cleaned_tokenized_test_y[i])
#
print(y_true[0:20])
print(y_pred[0:20])
#
i_pred = 0
for i in range(len(y_true)):
    if y_pred[i] in y_true[i]:
        i_pred += 1
    print("Predicted: " + str(y_pred[i]) + " --> " + str(y_true[i]))
print("\nAccuracy score: " + str(i_pred/len(y_true) * 100) + "%")

First 5 lines of training corpus: 
[['exalts', 'shall', 'humbled', 'humbles', 'shall', 'exaltedmatt', 'semst'], ['rt', 'prayerbullets', 'remove', 'nehushtan', 'previous', 'moves', 'god', 'become', 'idols', 'high', 'places', 'kings', 'semst'], ['brainman', 'heidtjj', 'benjaminlives', 'sought', 'truth', 'soul', 'found', 'strong', 'enough', 'stand', 'merits', 'semst'], ['god', 'utterly', 'powerless', 'without', 'human', 'intervention', 'semst'], ['david_cameron', 'miracles', 'multiculturalism', 'miracles', 'shady', 'taqiya', 'tawriya', 'jaziya', 'kafirs', 'dhimmi', 'jihad', 'allah', 'semst']]

First 5 lines of training corpus: 
[['anger', 'anticipation', 'disgust', 'joy', 'trust'], ['joy', 'sadness', 'trust'], ['anticipation', 'joy', 'trust'], ['anticipation', 'fear', 'sadness', 'trust'], ['anger', 'anticipation', 'fear', 'joy', 'surprise', 'trust']]

Shape of training corpus: 
(1956,)
Shape of training corpus: 
(1956,)

--------------------------------------

[['anger', 'anticipation', '